In [202]:
from nltk.corpus import stopwords
import string
from nltk.tokenize import RegexpTokenizer
import spacy
nlp = spacy.load("ru_core_news_sm")
import gensim.downloader
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import random
from gensim import models
from collections import Counter

In [2]:
tokenizer = RegexpTokenizer('\w+')

In [3]:
def lemmatize(text):
    while '»' in text:
        text = text.replace('»', '')
    while '«' in text:
        text = text.replace('«', '')
    while '(' in text:
        text = text.replace('(', '')
    while ')' in text:
        text = text.replace(')', '')

    symbs = ['—', "»", '«', 'note', '…']
    stop = set(stopwords.words('russian') + list(string.punctuation) + symbs)

    doc = nlp(text) 

    all_sentences = []
    for sent in doc.sents:
        tokens = [i for i in tokenizer.tokenize(str(sent).lower()) if i not in stop]

        lemmatized = []
        for word in tokens:
            lemma = morph.parse(word)[0].normal_form
            lemmatized.append(lemma)
        all_sentences.append(lemmatized)

    return all_sentences

In [4]:
with open('otcy_i_deti_1862.txt', 'r') as file:
    text = file.read()
words = lemmatize(text)

In [51]:
model = gensim.models.Word2Vec(
    window = 5,
    min_count=2,
    workers = 4
    )

In [52]:
model.build_vocab(words, progress_per=100)

In [54]:
model.corpus_count

4105

In [55]:
model.train(words, total_examples=model.corpus_count, epochs = model.epochs)

(133424, 163585)

In [56]:
model.save('testmodel')

In [234]:
for index, word in enumerate(model.wv.index_to_key):
    if index == 20:
        break
    print(f"word #{index}/{len(model.wv.index_to_key )} is {word}")

word #0/3576 is это
word #1/3576 is базар
word #2/3576 is петрович
word #3/3576 is аркадий
word #4/3576 is свой
word #5/3576 is который
word #6/3576 is павел
word #7/3576 is человек
word #8/3576 is говорить
word #9/3576 is николай
word #10/3576 is сказать
word #11/3576 is рука
word #12/3576 is знать
word #13/3576 is ты
word #14/3576 is сергеевич
word #15/3576 is промолвить
word #16/3576 is базаров
word #17/3576 is сам
word #18/3576 is глаз
word #19/3576 is анна


In [224]:
#Если нужно будет найти самые частые слова

# def most_common(text):
#     while '»' in text:
#         text = text.replace('»', '')
#     while '«' in text:
#         text = text.replace('«', '')
#     while '(' in text:
#         text = text.replace('(', '')
#     while ')' in text:
#         text = text.replace(')', '')

#     symbs = ['—', "»", '«', 'note', '…']
#     stop = set(stopwords.words('russian') + list(string.punctuation) + symbs)

#     words = []

#     tokens = [i for i in tokenizer.tokenize(text.lower()) if i not in stop]
#     for word in tokens:
#         lemma = morph.parse(word)[0].normal_form
#         words.append(lemma)

#     counter = Counter(words)
#     most_common_100 = counter.most_common(100)
#     most_common_100 = most_common_100[0:100]

#     common = []
#     for el in most_common_100:
#         common.append(el[0])

#     return common

In [233]:
def analogies(word):
    similar = model.wv.most_similar(word)[0][0]
    different = model.wv.most_similar(negative = word)[0][0]
    second_word = model.wv.most_similar(positive = [word, similar], negative = different)[0][0]

    analogy = random.choice(model.wv.index_to_key)
    second_analogy = model.wv.most_similar(positive = [analogy, similar], negative = different)[0][0]

    return f"От слова '{word}' до слова '{second_word}' вектор примерно такой же как от слова '{analogy}' до слова '{second_analogy}'"

In [231]:
for i in range(101):
    word = random.choice(model.wv.index_to_key)
    print(analogies(word))

От слова 'закапать' до слова 'страх' вектор примерно такой же как от слова 'казаться' до слова 'робеть'
От слова 'разъединение' до слова 'положить' вектор примерно такой же как от слова 'злость' до слова 'взять'
От слова 'сломать' до слова 'измениться' вектор примерно такой же как от слова 'имя' до слова 'тело'
От слова 'имение' до слова 'черта' вектор примерно такой же как от слова 'себя' до слова 'остаться'
От слова 'понять' до слова 'нога' вектор примерно такой же как от слова 'расписывать' до слова 'произносить'
От слова 'крепкий' до слова 'бал' вектор примерно такой же как от слова 'продать' до слова 'разговор'
От слова 'ходить' до слова 'особенный' вектор примерно такой же как от слова 'лес' до слова 'нога'
От слова 'колонна' до слова 'час' вектор примерно такой же как от слова 'видный' до слова 'час'
От слова 'прислуга' до слова 'приветливый' вектор примерно такой же как от слова 'бледно' до слова 'увидеться'
От слова 'вежливость' до слова 'исчезать' вектор примерно такой же как